In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [2]:

# 0. 기본 설정
N = 200  # 우리가 만들 데이터 개수
fake = Faker('ko_KR') # 한국어 기반 가짜 데이터 생성

print(f"{N}개의 더미 데이터를 생성합니다...")

200개의 더미 데이터를 생성합니다...


In [3]:
# --- 1. Product_Master (제품 마스터) ---
# 제품은 마스터 정보이므로, N개(200개)보다는 적은 게 현실적이지만,
# 연습을 위해 N개로 만듭니다.
print("1. Product_Master 생성 중...")
product_categories = ['세럼', '미스트', '크림', '선케어', '클렌저']
products = {
    'product_id': [f'dalba-prod-{i:03d}' for i in range(1, N + 1)],
    'product_name': [f'달바 {random.choice(product_categories)} {i}호' for i in range(1, N + 1)],
    'category': [random.choice(product_categories) for _ in range(N)],
    'price': np.random.randint(25000, 80000, N) // 100 * 100 # 100원 단위로
}
df_products = pd.DataFrame(products)
# 생성된 ID 리스트를 나중에 외래 키(FK)로 쓰기 위해 저장
product_id_list = df_products['product_id'].tolist()



1. Product_Master 생성 중...


In [4]:
# --- 2. Influencer_Master (인플루언서 마스터) ---
print("2. Influencer_Master 생성 중...")
platforms = ['Instagram', 'YouTube', 'TikTok']
inf_categories = ['뷰티', '라이프스타일', '패션', '여행', '푸드']

influencers = {
    'inf_id': [f'{fake.user_name()}_{random.choice(platforms).lower()}' for _ in range(N)],
    'inf_name': [fake.name() for _ in range(N)],
    'platform': [random.choice(platforms) for _ in range(N)],
    'follower_count': np.random.randint(5000, 1500000, N), # 팔로워 5천~150만
    'avg_engagement_rate': np.round(np.random.uniform(0.005, 0.15, N), 4), # 참여율 0.5% ~ 15%
    'main_category': [random.choice(inf_categories) for _ in range(N)],
    'estimated_cost_per_post': np.random.randint(50000, 10000000, N) // 10000 * 10000, # 1만원 단위
    'genai_brand_fit_score': np.round(np.random.uniform(1.0, 5.0, N), 1), # 1.0 ~ 5.0
    'genai_brand_fit_reason': [f"{random.choice(['고급진', '영한', '전문적인', '클린한'])} 이미지. {random.choice(['타겟 부합', '콘텐츠 퀄리티 우수', '최근 평판 좋음'])}." for _ in range(N)]
}
df_influencers = pd.DataFrame(influencers)
# 생성된 ID 리스트를 나중에 외래 키(FK)로 쓰기 위해 저장
inf_id_list = df_influencers['inf_id'].tolist()


2. Influencer_Master 생성 중...


In [5]:
# --- 3. Campaign_Master (캠페인 마스터) ---
print("3. Campaign_Master 생성 중...")
campaigns = {
    'campaign_id': [f'DALBA-CAMP-{i:03d}' for i in range(1, N + 1)],
    'campaign_name': [f'2025년 {i%4+1}분기 {random.choice(product_categories)} 런칭' for i in range(1, N + 1)],
    'product_id': [random.choice(product_id_list) for _ in range(N)], # [중요] 1번 테이블의 product_id에서 가져옴
    'start_date': [fake.date_between(start_date='-1y', end_date='today') for _ in range(N)],
    'total_budget': np.random.randint(10000000, 500000000, N) // 100000 * 100000
}
# 시작일로부터 30~90일 뒤를 종료일로 설정
df_campaigns = pd.DataFrame(campaigns)
df_campaigns['end_date'] = df_campaigns['start_date'].apply(lambda x: x + timedelta(days=random.randint(30, 90)))
# 생성된 ID 리스트를 나중에 외래 키(FK)로 쓰기 위해 저장
campaign_id_list = df_campaigns['campaign_id'].tolist()


3. Campaign_Master 생성 중...


In [ ]:
# --- 4. Campaign_Performance (캠페인 성과) ---
# 가장 중요한 '거래 내역' 테이블.
print("4. Campaign_Performance 생성 중...")
performances = []
for i in range(N):
    # 실제 성과 데이터는 훨씬 더 현실적인 로직이 필요함
    # (예: 캠페인 기간 내에 포스팅 날짜가 찍히도록)
    
    # 지금은 MVP를 위해 랜덤하게 생성
    chosen_inf_id = random.choice(inf_id_list)
    chosen_camp_id = random.choice(campaign_id_list)
    
    # 현실성을 위해 cost, impressions, clicks, conversions가 논리적으로 연결되게 만듦
    actual_cost = np.random.randint(50000, 5000000, 1)[0]
    impressions = np.random.randint(actual_cost // 100, actual_cost // 10, 1)[0] # 비용 대비 노출
    clicks = int(impressions * np.random.uniform(0.01, 0.05)) # 노출 대비 클릭 (CTR 1~5%)
    conversions = int(clicks * np.random.uniform(0.005, 0.03)) # 클릭 대비 전환 (CVR 0.5~3%)
    revenue = conversions * np.random.randint(30000, 60000, 1)[0] # 전환당 평균 매출
    
    performances.append({
        'perf_id': i + 1,
        'campaign_id': chosen_camp_id, # [중요] 3번 테이블의 ID
        'inf_id': chosen_inf_id,      # [중요] 2번 테이블의 ID
        'post_date': fake.date_between(start_date='-6m', end_date='today'),
        'post_url': f'https://instagram.com/p/{fake.password(length=11)}',
        'actual_cost': actual_cost,
        'impressions': impressions,
        'clicks': clicks,
        'conversions': conversions,
        'revenue': revenue,
        'genai_comment_summary': f"긍정 {random.randint(70, 95)}%. '{random.choice(['흡수력', '향기', '패키징', '광채'])}' 키워드 언급 다수."
    })

df_performance = pd.DataFrame(performances)

print("\n--- 모든 데이터 생성 완료! ---")

# --- 결과 확인 (각 DataFrame의 상위 5개 행 출력) ---
print("\n[Product_Master 샘플]")
print(df_products.head())
print("\n[Influencer_Master 샘플]")
print(df_influencers.head())
print("\n[Campaign_Master 샘플]")
print(df_campaigns.head())
print("\n[Campaign_Performance 샘플]")
print(df_performance.head())


4. Campaign_Performance 생성 중...

--- 모든 데이터 생성 완료! ---

[Product_Master 샘플]
       product_id product_name category  price
0  dalba-prod-001    달바 미스트 1호       크림  31900
1  dalba-prod-002     달바 크림 2호       세럼  28800
2  dalba-prod-003    달바 미스트 3호      클렌저  37300
3  dalba-prod-004     달바 세럼 4호      선케어  31100
4  dalba-prod-005     달바 크림 5호       세럼  64000

[Influencer_Master 샘플]
                inf_id inf_name   platform  follower_count  \
0   minseo53_instagram      김예진  Instagram          836940   
1    oeunjeong_youtube      이민준     TikTok           18712   
2       ggim_instagram      김정호  Instagram          914622   
3  eunyeongbag_youtube      김병철     TikTok         1020782   
4  yeonghyii_instagram      이승현    YouTube          835698   

   avg_engagement_rate main_category  estimated_cost_per_post  \
0               0.1397            푸드                  3050000   
1               0.0855            뷰티                  2980000   
2               0.1312            여행              

In [7]:
# --- (선택) CSV 파일로 저장 ---
df_products.to_csv('product_master.csv', index=False)
df_influencers.to_csv('influencer_master.csv', index=False)
df_campaigns.to_csv('campaign_master.csv', index=False)
df_performance.to_csv('campaign_performance.csv', index=False)

print("\nCSV 파일로 저장 완료!")


CSV 파일로 저장 완료!
